# INTELLIGENZA ARTIFICIALE
**Prof. Marco Zorzi, Dr. Alberto Testolin**

## Mini-Progetto Individuale - Riconoscimento Cifre Manoscritte

**Nome**: [Inserire nome]  
**Cognome**: [Inserire cognome]  
**Matricola**: [Inserire matricola]  
**Data di consegna**: [Inserire data]

---

### Obiettivo
Implementare simulazioni per studiare il riconoscimento di cifre manoscritte
utilizzando reti neurali Multi-Layer Perceptron (MLP) e Convolutional Neural Networks (CNN).

### Setup Ambiente
- Python version: Python 3.11.12
- TensorFlow: ✅ Disponibile
- PyTorch: ✅ Disponibile
- scikit-learn: ✅ Disponibile

### Punti da sviluppare:
- **[a]** Variazione neuroni/strati e iper-parametri [2 punti]
- **[b]** Cifre più difficili da riconoscere (matrice confusione) [1 punto]  
- **[c]** Curve psicometriche con rumore graduale [1 punto]
- **[d]** Riduzione dataset training (10%) [1 punto]
- **[e]** Miglioramento con rumore nel training [1 punto]
- **[bonus]** Estensione a FashionMNIST [punto bonus]

In [ ]:
# Import delle librerie necessarie
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import os
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD

# PyTorch (sempre disponibile)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Dataset
from torchvision.datasets import MNIST, FashionMNIST
import torchvision.transforms as transforms

# Configurazione per riproducibilità
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)

# Configurazione TensorFlow
tf.random.set_seed(RANDOM_STATE)
print(f"TensorFlow version: {tf.__version__}")

# Configurazione plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("🚀 Setup completato!")
print(f"NumPy version: {np.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"Scikit-learn version: {metrics.__version__}")

## Caricamento e Preprocessing dei Dati

Utilizziamo il dataset MNIST come base per tutti gli esperimenti.
Il dataset contiene 70.000 immagini di cifre manoscritte (60.000 training + 10.000 test).

In [ ]:
# Caricamento dataset MNIST
print("📊 Caricamento dataset MNIST...")

# Caricamento con torchvision
mnist_train = MNIST(root='./data', train=True, download=True)
mnist_test = MNIST(root='./data', train=False, download=True)

# Conversione in numpy arrays
X_train_full = mnist_train.data.numpy()
y_train_full = mnist_train.targets.numpy()
X_test = mnist_test.data.numpy() 
y_test = mnist_test.targets.numpy()

print(f"✅ Dataset caricato:")
print(f"   Training set: {X_train_full.shape[0]} immagini")
print(f"   Test set: {X_test.shape[0]} immagini")
print(f"   Dimensioni immagine: {X_train_full.shape[1]}x{X_train_full.shape[2]}")
print(f"   Numero classi: {len(np.unique(y_train_full))}")

### Preprocessing dei Dati

In [ ]:
# Preprocessing base
def preprocess_mnist(X_train, X_test, for_cnn=False):
    """
    Preprocessing del dataset MNIST
    
    Args:
        X_train, X_test: array numpy delle immagini
        for_cnn: se True, mantiene forma 2D per CNN, altrimenti flattens per MLP
    
    Returns:
        X_train_proc, X_test_proc: dati preprocessati
    """
    if for_cnn:
        # Per CNN: normalizza e aggiungi dimensione canale
        X_train_proc = X_train.astype('float32') / 255.0
        X_test_proc = X_test.astype('float32') / 255.0
        X_train_proc = X_train_proc.reshape(-1, 28, 28, 1)
        X_test_proc = X_test_proc.reshape(-1, 28, 28, 1)
    else:
        # Per MLP: flatten e normalizza
        X_train_proc = X_train.reshape(X_train.shape[0], -1).astype('float32') / 255.0
        X_test_proc = X_test.reshape(X_test.shape[0], -1).astype('float32') / 255.0
    
    return X_train_proc, X_test_proc

# Preprocessing iniziale per MLP
X_train_mlp, X_test_mlp = preprocess_mnist(X_train_full, X_test, for_cnn=False)

# Preprocessing per CNN (useremo dopo)
X_train_cnn, X_test_cnn = preprocess_mnist(X_train_full, X_test, for_cnn=True)

print("✅ Preprocessing completato")
print(f"   MLP format: {X_train_mlp.shape}")
print(f"   CNN format: {X_train_cnn.shape}")

### Visualizzazione Campioni del Dataset

In [ ]:
# Visualizzazione esempi del dataset
def plot_mnist_samples(X, y, n_samples=10, title="Campioni MNIST"):
    """Visualizza campioni casuali del dataset"""
    
    fig, axes = plt.subplots(2, 5, figsize=(12, 6))
    fig.suptitle(title, fontsize=16)
    
    # Seleziona campioni casuali
    indices = np.random.choice(len(X), n_samples, replace=False)
    
    for i, idx in enumerate(indices):
        ax = axes[i//5, i%5]
        
        # Reshape per visualizzazione se necessario
        img = X[idx].reshape(28, 28) if len(X[idx].shape) == 1 else X[idx].squeeze()
        
        ax.imshow(img, cmap='gray')
        ax.set_title(f'Cifra: {y[idx]}')
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()

# Mostra campioni del dataset
plot_mnist_samples(X_train_full, y_train_full, title="Campioni Dataset MNIST")

## PUNTO A: Analisi Architetturale [2 punti]

Studiamo l'effetto del numero di neuroni e strati nascosti sulle prestazioni 
di MLP (scikit-learn) e CNN (PyTorch/TensorFlow).

### Nota Implementativa
Useremo sia TensorFlow che PyTorch per CNN

In [ ]:
# TODO: Implementare analisi sistematica delle architetture
print("📋 Punto A: Da implementare - Analisi architetturale")
print("   🔧 MLP: scikit-learn (sempre disponibile)")
print("   🔧 CNN: PyTorch + TensorFlow")

## Funzioni Utility per CNN con PyTorch

Implementiamo le funzioni base per CNN con PyTorch che funzionano sempre.

In [ ]:
class SimpleCNN(nn.Module):
    """CNN semplice con PyTorch"""
    
    def __init__(self, num_filters=32, hidden_size=128):
        super(SimpleCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(1, num_filters, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(num_filters, num_filters*2, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)
        
        # Calcolo dimensioni per il layer fully connected
        self.fc1 = nn.Linear(num_filters*2 * 7 * 7, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 10)
        
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.dropout(x)
        x = x.view(-1, x.size(1) * x.size(2) * x.size(3))
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

def train_pytorch_cnn(model, train_loader, test_loader, epochs=5, lr=0.001):
    """Training di CNN con PyTorch"""
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    train_losses = []
    test_accuracies = []
    
    for epoch in range(epochs):
        # Training
        model.train()
        running_loss = 0.0
        
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        # Evaluation
        model.eval()
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        accuracy = 100 * correct / total
        train_losses.append(running_loss / len(train_loader))
        test_accuracies.append(accuracy)
        
        print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%')
    
    return train_losses, test_accuracies

print("✅ Funzioni PyTorch CNN definite")

## PUNTO B: Analisi degli Errori MLP [1 punto]

Identifichiamo le cifre più difficili da riconoscere utilizzando la matrice di confusione.

In [ ]:
# TODO: Implementare analisi errori MLP
print("📋 Punto B: Da implementare - Analisi errori")

## PUNTO C: Curve Psicometriche [1 punto]

Studiamo come cambia l'accuratezza con l'introduzione graduale di rumore.

In [ ]:
# TODO: Implementare curve psicometriche
print("📋 Punto C: Da implementare - Curve psicometriche")

## PUNTO D: Training Set Ridotto [1 punto]

Analizziamo l'effetto della riduzione drastica dei pattern di training (10%).

In [ ]:
# TODO: Implementare training con dataset ridotto
print("📋 Punto D: Da implementare - Dataset ridotto")

## PUNTO E: Training con Rumore [1 punto]

Miglioriamo l'accuratezza sui pattern rumorosi introducendo rumore nel training.

In [ ]:
# TODO: Implementare training con rumore
print("📋 Punto E: Da implementare - Training con rumore")

## PUNTO BONUS: Estensione a FashionMNIST

Estendiamo le simulazioni al dataset FashionMNIST.

In [ ]:
# TODO: Implementare estensione FashionMNIST
print("📋 Punto Bonus: Da implementare - FashionMNIST")

## Conclusioni

### Riassunto Risultati
[Da completare con i risultati degli esperimenti]

### Discussione
[Da completare con interpretazioni teoriche]

### Limitazioni e Sviluppi Futuri
[Da completare]

In [ ]:
print("🎯 Template progetto creato!")
print("📝 Prossimi passi:")
print("   1. Implementare Punto A: Analisi architetturale")
print("   2. Implementare Punto B: Analisi errori")
print("   3. Implementare Punto C: Curve psicometriche")
print("   4. Implementare Punto D: Dataset ridotto") 
print("   5. Implementare Punto E: Training con rumore")
print("   6. [Opzionale] Punto Bonus: FashionMNIST")
print("")
print("💡 Framework disponibili:")
print("   ✅ scikit-learn (MLP)")
print("   ✅ PyTorch (CNN)")
print("   ✅ TensorFlow (CNN)")